In [3]:
from azureml.core import Workspace, Experiment

ws = Workspace.from_config()
exp = Experiment(workspace=ws, name="udacity-project")

print('Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      'Subscription id: ' + ws.subscription_id, 
      'Resource group: ' + ws.resource_group, sep = '\n')

run = exp.start_logging()

Workspace name: quick-starts-ws-238984
Azure region: southcentralus
Subscription id: 9b72f9e6-56c5-4c16-991b-19c652994860
Resource group: aml-quickstarts-238984


In [4]:
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException

cluster_name = "Udacity"

# TODO: Create compute cluster
# Use vm_size = "Standard_D2_V2" in your provisioning configuration.
# max_nodes should be no greater than 4.

try:
    cluster = ComputeTarget(workspace=ws, name=cluster_name)
    print("Found existing cluster")
except ComputeTargetException:
    print("Creating new cluster")

    compute_config = AmlCompute.provisioning_configuration(
        vm_size = 'STANDARD_D2_V2',
        min_modes=0,
        max_nodes=4)
    cluster = ComputeTarget.create(ws, cluster_name, compute_config)
cluster.wait_for_completion(show_output=True)

Found existing cluster

Running


In [5]:
from azureml.widgets import RunDetails
from azureml.train.sklearn import SKLearn
from azureml.train.hyperdrive.run import PrimaryMetricGoal
from azureml.train.hyperdrive.policy import BanditPolicy
from azureml.train.hyperdrive.sampling import RandomParameterSampling
from azureml.train.hyperdrive.runconfig import HyperDriveConfig
from azureml.train.hyperdrive.parameter_expressions import choice, uniform
from azureml.core import Environment, ScriptRunConfig
import os

# Specify parameter sampler
ps = RandomParameterSampling(
    {
        '--C': choice(0.1,0.5,1,5),
        '--max_iter': choice(50, 100, 500)
    }
)

# Specify a Policy
policy = BanditPolicy(evaluation_interval=2, slack_factor=0.1)

if "training" not in os.listdir():
    os.mkdir("./training")

# Setup environment for your training run
sklearn_env = Environment.from_conda_specification(name='sklearn-env', file_path='conda_dependencies.yml')

# Create a ScriptRunConfig Object to specify the configuration details of your training job
src = ScriptRunConfig(source_directory='./',
                script='train_wv.py',
                compute_target='Udacity',
                environment=sklearn_env)

#primary_metric = 'Accuracy'
# Create a HyperDriveConfig using the src object, hyperparameter sampler, and policy.
hyperdrive_config = HyperDriveConfig(run_config=src,
                hyperparameter_sampling=ps,
                policy=policy,
                primary_metric_name='Accuracy',
                primary_metric_goal=PrimaryMetricGoal.MAXIMIZE,
                max_total_runs=100,
                max_concurrent_runs=4)

In [6]:
# Submit your hyperdrive run to the experiment and show run details with the widget.

hyperdrive_run = exp.submit(config=hyperdrive_config)
RunDetails(hyperdrive_run).show()
hyperdrive_run.wait_for_completion(show_output=True)

_HyperDriveWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO'…

RunId: HD_ed85bbbb-1b68-43a6-bb97-afa38d5224ff
Web View: https://ml.azure.com/runs/HD_ed85bbbb-1b68-43a6-bb97-afa38d5224ff?wsid=/subscriptions/9b72f9e6-56c5-4c16-991b-19c652994860/resourcegroups/aml-quickstarts-238984/workspaces/quick-starts-ws-238984&tid=660b3398-b80e-49d2-bc5b-ac1dc93b5254

Streaming azureml-logs/hyperdrive.txt

[2023-07-20T09:55:02.476187][GENERATOR][INFO]Trying to sample '4' jobs from the hyperparameter space
[2023-07-20T09:55:03.0248387Z][SCHEDULER][INFO]Scheduling job, id='HD_ed85bbbb-1b68-43a6-bb97-afa38d5224ff_0' 
[2023-07-20T09:55:03.1744594Z][SCHEDULER][INFO]Scheduling job, id='HD_ed85bbbb-1b68-43a6-bb97-afa38d5224ff_1' 
[2023-07-20T09:55:03.3998458Z][SCHEDULER][INFO]Scheduling job, id='HD_ed85bbbb-1b68-43a6-bb97-afa38d5224ff_3' 
[2023-07-20T09:55:03.359325][GENERATOR][INFO]Successfully sampled '4' jobs, they will soon be submitted to the execution target.
[2023-07-20T09:55:03.5246192Z][SCHEDULER][INFO]Scheduling job, id='HD_ed85bbbb-1b68-43a6-bb97-afa38d5224

{'runId': 'HD_ed85bbbb-1b68-43a6-bb97-afa38d5224ff',
 'target': 'Udacity',
 'status': 'Completed',
 'startTimeUtc': '2023-07-20T09:55:01.552292Z',
 'endTimeUtc': '2023-07-20T10:12:37.277541Z',
 'services': {},
 'properties': {'primary_metric_config': '{"name":"Accuracy","goal":"maximize"}',
  'resume_from': 'null',
  'runTemplate': 'HyperDrive',
  'azureml.runsource': 'hyperdrive',
  'platform': 'AML',
  'ContentSnapshotId': '41291924-f705-4339-b15f-7f2b4c5be4a6',
  'user_agent': 'python/3.8.5 (Linux-5.15.0-1040-azure-x86_64-with-glibc2.10) msrest/0.7.1 Hyperdrive.Service/1.0.0 Hyperdrive.SDK/core.1.51.0',
  'space_size': '12',
  'score': '0.9135712551590192',
  'best_child_run_id': 'HD_ed85bbbb-1b68-43a6-bb97-afa38d5224ff_0',
  'best_metric_status': 'Succeeded',
  'best_data_container_id': 'dcid.HD_ed85bbbb-1b68-43a6-bb97-afa38d5224ff_0'},
 'inputDatasets': [],
 'outputDatasets': [],
 'runDefinition': {'configuration': None,
  'attribution': None,
  'telemetryValues': {'amlClientType'

In [9]:
import joblib
# Get your best run and save the model from that run.

best_run = hyperdrive_run.get_best_run_by_primary_metric()
print(best_run)
best_run.register_model(model_name = 'best_hd_model.pkl', model_path='outputs/model.joblib')

Run(Experiment: udacity-project,
Id: HD_ed85bbbb-1b68-43a6-bb97-afa38d5224ff_0,
Type: azureml.scriptrun,
Status: Completed)


Model(workspace=Workspace.create(name='quick-starts-ws-238984', subscription_id='9b72f9e6-56c5-4c16-991b-19c652994860', resource_group='aml-quickstarts-238984'), name=best_hd_model.pkl, id=best_hd_model.pkl:1, version=1, tags={}, properties={})

In [10]:
print(best_run.get_details()['runDefinition']['arguments'])

['--C', '5', '--max_iter', '500']


In [11]:
print(best_run.get_metrics())

{'Regularization Strength:': 5.0, 'Max iterations:': 500, 'Accuracy': 0.9135712551590192}


In [12]:
from azureml.data.dataset_factory import TabularDatasetFactory

# Create TabularDataset using TabularDatasetFactory
# Data is available at: 
# "https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv"

data = TabularDatasetFactory.from_delimited_files("https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv")

In [13]:

from train_wv import clean_data
import pandas as pd

# Use the clean_data function to clean your data.
x, y = clean_data(data)

data_joined = pd.concat([x, y], axis=1)

In [15]:
from azureml.train.automl import AutoMLConfig

# Set parameters for AutoMLConfig
# NOTE: DO NOT CHANGE THE experiment_timeout_minutes PARAMETER OR YOUR INSTANCE WILL TIME OUT.
# If you wish to run the experiment longer, you will need to run this notebook in your own
# Azure tenant, which will incur personal costs.
automl_config = AutoMLConfig(
    experiment_timeout_minutes=30,
    task='classification',
    primary_metric='accuracy',
    training_data=data_joined,
    label_column_name='y',
    n_cross_validations=2)

In [16]:
# Submit your automl run

experiment = Experiment(ws, 'automl')
run = experiment.submit(config=automl_config, show_out=True)

2023-07-20:10:20:08,56 WARNING  [local_experiment_driver.py:142] Running on local machine. Note that local runs always run synchronously even if you use the parameter 'show_output=False'


Experiment,Id,Type,Status,Details Page,Docs Page
automl,AutoML_554b58e0-0e72-4169-ae13-28e61484af3d,automl,Preparing,Link to Azure Machine Learning studio,Link to Documentation


2023/07/20 10:27:07 WARNING mlflow.sklearn: Model was missing function: predict. Not logging python_function flavor!
2023-07-20:10:53:56,889 INFO     [explanation_client.py:334] Using default datastore for uploads


In [17]:
# Retrieve and save your best automl model.

best_auto_run, model = run.get_output()
best_auto_run.register_model(model_name = 'automl_best_run', model_path='outputs/')
print(best_auto_run)

Run(Experiment: automl,
Id: AutoML_554b58e0-0e72-4169-ae13-28e61484af3d_30,
Type: None,
Status: Completed)


In [21]:
print(best_auto_run.get_details())

{'runId': 'AutoML_554b58e0-0e72-4169-ae13-28e61484af3d_30', 'target': 'local', 'status': 'Completed', 'startTimeUtc': '2023-07-20T10:50:34.338255Z', 'endTimeUtc': '2023-07-20T10:51:12.919188Z', 'services': {}, 'properties': {'runTemplate': 'automl_child', 'pipeline_id': '__AutoML_Ensemble__', 'pipeline_spec': '{"pipeline_id":"__AutoML_Ensemble__","objects":[{"module":"azureml.train.automl.ensemble","class_name":"Ensemble","spec_class":"sklearn","param_args":[],"param_kwargs":{"automl_settings":"{\'task_type\':\'classification\',\'primary_metric\':\'accuracy\',\'verbosity\':20,\'ensemble_iterations\':15,\'is_timeseries\':False,\'name\':\'automl\',\'compute_target\':\'local\',\'subscription_id\':\'9b72f9e6-56c5-4c16-991b-19c652994860\',\'region\':\'southcentralus\',\'spark_service\':None}","ensemble_run_id":"AutoML_554b58e0-0e72-4169-ae13-28e61484af3d_30","experiment_name":null,"workspace_name":"quick-starts-ws-238984","subscription_id":"9b72f9e6-56c5-4c16-991b-19c652994860","resource_gr

In [20]:
print(best_auto_run.get_metrics())

{'accuracy': 0.9159939301972686, 'matthews_correlation': 0.5234661199900245, 'recall_score_micro': 0.9159939301972686, 'AUC_macro': 0.9481029197629949, 'precision_score_weighted': 0.9073896486431294, 'balanced_accuracy': 0.7222707872471634, 'precision_score_micro': 0.9159939301972686, 'norm_macro_recall': 0.44454157449432696, 'AUC_weighted': 0.9481029197901383, 'weighted_accuracy': 0.9641136984773933, 'f1_score_weighted': 0.909185816866413, 'recall_score_macro': 0.7222707872471634, 'log_loss': 0.2283384159298979, 'average_precision_score_weighted': 0.9554944221944566, 'precision_score_macro': 0.808564839453568, 'average_precision_score_macro': 0.824776034372283, 'f1_score_micro': 0.9159939301972686, 'f1_score_macro': 0.7554376635491381, 'AUC_micro': 0.9806795065867491, 'average_precision_score_micro': 0.9814720705348196, 'recall_score_weighted': 0.9159939301972686, 'confusion_matrix': 'aml://artifactId/ExperimentRun/dcid.AutoML_554b58e0-0e72-4169-ae13-28e61484af3d_30/confusion_matrix',

In [22]:
cluster.delete()